In [1]:
import json
import os
import pickle
import time

from src.models import TrafficModel
from tqdm import tqdm

## Load distributions

### Demography distributions

In [2]:
data_dir = 'input_data/base_distributions/demography/'

data_file = 'population_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    population_dist = json.load(f)

data_file = 'demography_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    demography_dist = json.load(f)


### Decision tree distributions

In [3]:
data_dir = 'input_data/base_distributions/decision_tree/'

data_file = 'pub_trans_comfort_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    pub_trans_comfort_dist = json.load(f)

data_file = 'pub_trans_punctuality_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    pub_trans_punctuality_dist = json.load(f)

data_file = 'bicycle_infrastr_comfort_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    bicycle_infrastr_comfort_dist = json.load(f)

data_file = 'pedestrian_inconvenience_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    pedestrian_inconvenience_dist = json.load(f)

data_file = 'household_persons_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    household_persons_dist = json.load(f)

data_file = 'household_cars_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    household_cars_dist = json.load(f)

data_file = 'household_bicycles_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    household_bicycles_dist = json.load(f)

### Travel planning distributions

In [4]:
data_dir = 'input_data/base_distributions/travel_planning/'

data_file = 'travels_num_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    travels_num_dist = json.load(f)

data_file = 'start_hour_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    start_hour_dist = json.load(f)

data_file = 'dest_type_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    dest_type_dist = json.load(f)

data_file = 'gravity_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    gravity_dist = json.load(f)

data_file = 'drivers_dist.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    drivers_dist = json.load(f)

## Load interregional distances and decision tree classifier

In [5]:
data_dir = 'input_data/'

# interregional distances
data_file = 'interregional_distances.json'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'r') as f:
    interregional_distances = json.load(f)


# decision tree
data_file = 'decision_tree.pickle'
data_path = os.path.join(data_dir, data_file)
with open(data_path, 'rb') as f:
    decision_tree = pickle.load(f)

## Spend time distribution

In [6]:
spend_time_dist_params = {
    "0-5": {
        "dom": {
            "loc": 120,
            "scale": 60
        },
        "praca": {
            "loc": 480,
            "scale": 60
        },
        "szkola": {
            "loc": 390,
            "scale": 120
        },
        "uczelnia": {
            "loc": 390,
            "scale": 120
        },
        "inne": {
            "loc": 120,
            "scale": 60
        },
    },
    "6-15_K": {
        "dom": {
            "loc": 120,
            "scale": 60
        },
        "praca": {
            "loc": 480,
            "scale": 60
        },
        "szkola": {
            "loc": 390,
            "scale": 120
        },
        "uczelnia": {
            "loc": 390,
            "scale": 120
        },
        "inne": {
            "loc": 120,
            "scale": 60
        },
    },
    "6-15_M": {
        "dom": {
            "loc": 120,
            "scale": 60
        },
        "praca": {
            "loc": 480,
            "scale": 60
        },
        "szkola": {
            "loc": 390,
            "scale": 120
        },
        "uczelnia": {
            "loc": 390,
            "scale": 120
        },
        "inne": {
            "loc": 120,
            "scale": 60
        },
    },
    "16-19_K": {
        "dom": {
            "loc": 120,
            "scale": 60
        },
        "praca": {
            "loc": 480,
            "scale": 60
        },
        "szkola": {
            "loc": 390,
            "scale": 120
        },
        "uczelnia": {
            "loc": 390,
            "scale": 120
        },
        "inne": {
            "loc": 120,
            "scale": 60
        },
    },
    "16-19_M": {
        "dom": {
            "loc": 120,
            "scale": 60
        },
        "praca": {
            "loc": 480,
            "scale": 60
        },
        "szkola": {
            "loc": 390,
            "scale": 120
        },
        "uczelnia": {
            "loc": 390,
            "scale": 120
        },
        "inne": {
            "loc": 120,
            "scale": 60
        },
    },
    "20-24_K": {
        "dom": {
            "loc": 120,
            "scale": 60
        },
        "praca": {
            "loc": 480,
            "scale": 60
        },
        "szkola": {
            "loc": 390,
            "scale": 120
        },
        "uczelnia": {
            "loc": 390,
            "scale": 120
        },
        "inne": {
            "loc": 120,
            "scale": 60
        },
    },
    "20-24_M": {
        "dom": {
            "loc": 120,
            "scale": 60
        },
        "praca": {
            "loc": 480,
            "scale": 60
        },
        "szkola": {
            "loc": 390,
            "scale": 120
        },
        "uczelnia": {
            "loc": 390,
            "scale": 120
        },
        "inne": {
            "loc": 120,
            "scale": 60
        },
    },
    "25-44_K": {
        "dom": {
            "loc": 120,
            "scale": 60
        },
        "praca": {
            "loc": 480,
            "scale": 60
        },
        "szkola": {
            "loc": 390,
            "scale": 120
        },
        "uczelnia": {
            "loc": 390,
            "scale": 120
        },
        "inne": {
            "loc": 120,
            "scale": 60
        },
    },
    "25-44_M": {
        "dom": {
            "loc": 120,
            "scale": 60
        },
        "praca": {
            "loc": 480,
            "scale": 60
        },
        "szkola": {
            "loc": 390,
            "scale": 120
        },
        "uczelnia": {
            "loc": 390,
            "scale": 120
        },
        "inne": {
            "loc": 120,
            "scale": 60
        },
    },
    "45-60_K": {
        "dom": {
            "loc": 120,
            "scale": 60
        },
        "praca": {
            "loc": 480,
            "scale": 60
        },
        "szkola": {
            "loc": 390,
            "scale": 120
        },
        "uczelnia": {
            "loc": 390,
            "scale": 120
        },
        "inne": {
            "loc": 120,
            "scale": 60
        },
    },
    "45-65_M": {
        "dom": {
            "loc": 120,
            "scale": 60
        },
        "praca": {
            "loc": 480,
            "scale": 60
        },
        "szkola": {
            "loc": 390,
            "scale": 120
        },
        "uczelnia": {
            "loc": 390,
            "scale": 120
        },
        "inne": {
            "loc": 120,
            "scale": 60
        },
    },
    "61-x_K": {
        "dom": {
            "loc": 120,
            "scale": 60
        },
        "praca": {
            "loc": 480,
            "scale": 60
        },
        "szkola": {
            "loc": 390,
            "scale": 120
        },
        "uczelnia": {
            "loc": 390,
            "scale": 120
        },
        "inne": {
            "loc": 120,
            "scale": 60
        },
    },
    "66-x_M": {
        "dom": {
            "loc": 120,
            "scale": 60
        },
        "praca": {
            "loc": 480,
            "scale": 60
        },
        "szkola": {
            "loc": 390,
            "scale": 120
        },
        "uczelnia": {
            "loc": 390,
            "scale": 120
        },
        "inne": {
            "loc": 120,
            "scale": 60
        },
    }
}

## TrafficModel test

In [7]:
start_timer = time.time()

In [8]:
start_time = 4 * 60
step_time = 60
end_time = 23 * 60


model = TrafficModel(
    N=600000,  # 635701
    population_dist=population_dist,
    demography_dist=demography_dist,
    pub_trans_comfort_dist=pub_trans_comfort_dist,
    pub_trans_punctuality_dist=pub_trans_punctuality_dist,
    bicycle_infrastr_comfort_dist=bicycle_infrastr_comfort_dist,
    pedestrian_inconvenience_dist=pedestrian_inconvenience_dist,
    household_persons_dist=household_persons_dist,
    household_cars_dist=household_cars_dist,
    household_bicycles_dist=household_bicycles_dist,
    travels_num_dist=travels_num_dist,
    start_hour_dist=start_hour_dist,
    dest_type_dist=dest_type_dist,
    spend_time_dist_params=spend_time_dist_params,
    decision_tree=decision_tree,
    gravity_dist=gravity_dist,
    drivers_dist=drivers_dist,
    interregional_distances=interregional_distances,
    start_time=start_time,
    step_time=step_time,
    end_time=end_time
)

In [9]:
for i in tqdm(range(start_time, end_time+1, step_time)):
    model.step()

print()
print(time.time() - start_timer)

100%|██████████| 20/20 [03:15<00:00,  9.76s/it]
409.75140929222107



In [10]:
model.agent_data_collector.get_agent_vars_dataframe().head(30)

agent_id home_region  age_sex  pub_trans_comfort  \
Step AgentID                                                     
0    0               0          61  45-65_M                3.0   
     1               1          35   66-x_M                3.0   
     2               2         304      0-5                NaN   
     3               3         138   61-x_K                4.0   
     4               4         261      0-5                NaN   
     5               5         269   66-x_M                3.0   
     6               6         219  20-24_M                3.0   
     7               7          40  25-44_K                3.0   
     8               8         143   66-x_M                3.0   
     9               9          22  45-60_K                3.0   
     10             10          26   61-x_K                3.0   
     11             11         357   61-x_K                3.0   
     12             12         217  45-60_K                4.0   
     13             13         164  25-44_K                2.0   
     14             14         363   61-x_K                3.0   
     15             15          33  20-24_K                3.0   
     16             16         368   61-x_K                3.0   
     17             17         343  25-44_K                2.0   
     18             18         135  45-65_M                3.0   
     19             19         103  25-44_K                3.0   
     20             20         158   66-x_M                4.0   
     21             21         343   61-x_K                3.0   
     22             22         195   66-x_M                3.0   
     23             23         333   66-x_M                2.0   
     24             24           2  25-44_K                3.0   
     25             25          40   66-x_M                4.0   
     26             26         332      0-5                NaN   
     27             27          15  25-44_M                2.0   
     28             28          95  25-44_K                2.0   
     29             29         158  25-44_M                2.0   

              pub_trans_punctuality  bicycle_infrastr_comfort  \
Step AgentID                                                    
0    0                          4.0                       4.0   
     1                          4.0                       2.0   
     2                          NaN                       NaN   
     3                          4.0                       0.0   
     4                          NaN                       NaN   
     5                          2.0                       2.0   
     6                          3.0                       3.0   
     7                          2.0                       2.0   
     8                          2.0                       0.0   
     9                          3.0                       1.0   
     10                         3.0                       2.0   
     11                         3.0                       2.0   
     12                         2.0                       2.0   
     13                         0.0                       4.0   
     14                         4.0                       2.0   
     15                         2.0                       4.0   
     16                         2.0                       1.0   
     17                         2.0                       3.0   
     18                         3.0                       2.0   
     19                         3.0                       3.0   
     20                         2.0                       4.0   
     21                         2.0                       4.0   
     22                         2.0                       2.0   
     23                         2.0                       1.0   
     24                         3.0                       2.0   
     25                         2.0                       2.0   
     26                         NaN                       NaN   
     27         

In [11]:
model.travels_data_collector.get_agent_vars_dataframe().head(50)

agent_id         start_region          start_place_type  \
Step AgentID                                                            
20   0               0            [61, 352]              [dom, praca]   
     1               1            [35, 318]               [dom, inne]   
     2               2                   []                        []   
     3               3           [138, 355]               [dom, inne]   
     4               4                   []                        []   
     5               5                   []                        []   
     6               6           [219, 217]              [dom, praca]   
     7               7            [40, 104]              [dom, praca]   
     8               8           [143, 147]               [dom, inne]   
     9               9            [22, 135]               [dom, inne]   
     10             10            [26, 100]               [dom, inne]   
     11             11           [357, 357]               [dom, inne]   
     12             12                   []                        []   
     13             13                   []                        []   
     14             14                   []                        []   
     15             15             [33, 67]           [dom, uczelnia]   
     16             16           [368, 268]               [dom, inne]   
     17             17             [343, 2]           [dom, uczelnia]   
     18             18                   []                        []   
     19             19            [103, 22]               [dom, inne]   
     20             20                   []                        []   
     21             21            [343, 36]               [dom, inne]   
     22             22                   []                        []   
     23             23           [333, 363]               [dom, inne]   
     24             24              [2, 16]              [dom, praca]   
     25             25             [40, 37]               [dom, inne]   
     26             26                   []                        []   
     27             27                   []                        []   
     28             28             [95, 95]               [dom, inne]   
     29             29            [158, 49]              [dom, praca]   
     30             30             [65, 18]              [dom, praca]   
     31             31  [205, 65, 205, 155]  [dom, praca, dom, praca]   
     32             32              [57, 8]               [dom, inne]   
     33             33           [163, 163]               [dom, inne]   
     34             34                   []                        []   
     35             35                   []                        []   
     36             36                   []                        []   
     37             37      [108, 267, 300]         [dom, inne, inne]   
     38             38           [341, 363]               [dom, inne]   
     39             39                   []                        []   
     40             40                   []                        []   
     41             41                   []                        []   
     42             42            [368, 38]           [dom, uczelnia]   
     43             43            [50, 126]              [dom, praca]   
     44             44                   []                        []   
     45             45                   []                        []   
     46             46              [36, 1]              [dom, praca]   
     47             47                   []                        []   
     48             48                   []                        []   
     49             49           [181, 154]             [dom, szkola]   

                      dest_region           dest_place_type  \
Step AgentID                                                  
20   0                  [352, 61]              [praca, dom]   
     1                  [318

In [12]:
df = model.agent_data_collector.get_agent_vars_dataframe()

In [17]:
d = df['age_sex'].value_counts().to_dict()
dict(sorted(d.items(), key=lambda item: item[1], reverse=True))

{'25-44_K': 95904,
 '25-44_M': 95716,
 '61-x_K': 91852,
 '45-65_M': 72309,
 '45-60_K': 58791,
 '66-x_M': 51670,
 '0-5': 40361,
 '6-15_M': 26700,
 '6-15_K': 26263,
 '20-24_M': 11903,
 '20-24_K': 11818,
 '16-19_K': 8405,
 '16-19_M': 8308}

In [18]:
dict(sorted(demography_dist.items(), key=lambda item: item[1], reverse=True))

{'25-44_K': 0.15951384048615952,
 '25-44_M': 0.15951384048615952,
 '61-x_K': 0.1528868871131129,
 '45-65_M': 0.12049717950282052,
 '45-60_K': 0.0985886014113986,
 '66-x_M': 0.085998874001126,
 '0-5': 0.06757193242806758,
 '6-15_K': 0.044112955887044114,
 '6-15_M': 0.044112955887044114,
 '20-24_K': 0.019773980226019776,
 '20-24_M': 0.019773980226019776,
 '16-19_K': 0.013827486172513828,
 '16-19_M': 0.013827486172513828}

In [23]:
df.columns

Index(['agent_id', 'home_region', 'age_sex', 'pub_trans_comfort',
       'pub_trans_punctuality', 'bicycle_infrastr_comfort',
       'pedestrian_inconvenience', 'household_persons', 'household_cars',
       'household_bicycles', 'travels_num'],
      dtype='object')

In [26]:
df['travels_num'].value_counts()

2    366311
0    176210
3     27057
4     25255
5      3151
6      2016
Name: travels_num, dtype: int64